In [27]:
from collections import deque, defaultdict
import itertools

class Order:
    def __init__(self, order_id, side, price, qty, timestamp):
        self.order_id = order_id
        self.side = side
        self.price = price
        self.qty = qty
        self.timestamp = timestamp

    def __repr__(self):
        return f"Order(id={self.order_id}, side={self.side}, price={self.price}, qty={self.qty})"


class OrderBook:
    def __init__(self):
        self.bids = defaultdict(deque)
        self.asks = defaultdict(deque)
        self.order_id_counter = itertools.count(1)
        self.timestamp = 0
        self.trades = []

    def best_bid(self):
        return max(self.bids.keys()) if self.bids else None

    def best_ask(self):
        return min(self.asks.keys()) if self.asks else None

    # ---------------- LIMIT ORDERS ----------------

    def add_limit_order(self, side, price, qty):
        self.timestamp += 1
        order_id = next(self.order_id_counter)
        incoming = Order(order_id, side, price, qty, self.timestamp)

        if side == "buy":
            self._match_buy(incoming)
            if incoming.qty > 0:
                self.bids[price].append(incoming)

        elif side == "sell":
            self._match_sell(incoming)
            if incoming.qty > 0:
                self.asks[price].append(incoming)

        else:
            raise ValueError("Side must be 'buy' or 'sell'")

        return incoming

    # ---------------- MARKET ORDERS ----------------

    def add_market_order(self, side, qty):
        self.timestamp += 1
        order_id = next(self.order_id_counter)
        market_order = Order(order_id, side, None, qty, self.timestamp)

        if side == "buy":
            while market_order.qty > 0 and self.asks:
                best_ask_price = self.best_ask()
                resting = self.asks[best_ask_price][0]

                trade_qty = min(market_order.qty, resting.qty)
                market_order.qty -= trade_qty
                resting.qty -= trade_qty

                self.trades.append(("BUY", best_ask_price, trade_qty))

                if resting.qty == 0:
                    self.asks[best_ask_price].popleft()
                    if not self.asks[best_ask_price]:
                        del self.asks[best_ask_price]

        elif side == "sell":
            while market_order.qty > 0 and self.bids:
                best_bid_price = self.best_bid()
                resting = self.bids[best_bid_price][0]

                trade_qty = min(market_order.qty, resting.qty)
                market_order.qty -= trade_qty
                resting.qty -= trade_qty

                self.trades.append(("SELL", best_bid_price, trade_qty))

                if resting.qty == 0:
                    self.bids[best_bid_price].popleft()
                    if not self.bids[best_bid_price]:
                        del self.bids[best_bid_price]

        else:
            raise ValueError("Side must be 'buy' or 'sell'")

    # ---------------- MATCHING HELPERS ----------------

    def _match_buy(self, buy_order):
        while buy_order.qty > 0 and self.asks:
            best_ask_price = self.best_ask()
            if best_ask_price > buy_order.price:
                break

            resting = self.asks[best_ask_price][0]
            trade_qty = min(buy_order.qty, resting.qty)

            buy_order.qty -= trade_qty
            resting.qty -= trade_qty
            self.trades.append(("BUY", best_ask_price, trade_qty))

            if resting.qty == 0:
                self.asks[best_ask_price].popleft()
                if not self.asks[best_ask_price]:
                    del self.asks[best_ask_price]

    def _match_sell(self, sell_order):
        while sell_order.qty > 0 and self.bids:
            best_bid_price = self.best_bid()
            if best_bid_price < sell_order.price:
                break

            resting = self.bids[best_bid_price][0]
            trade_qty = min(sell_order.qty, resting.qty)

            sell_order.qty -= trade_qty
            resting.qty -= trade_qty
            self.trades.append(("SELL", best_bid_price, trade_qty))

            if resting.qty == 0:
                self.bids[best_bid_price].popleft()
                if not self.bids[best_bid_price]:
                    del self.bids[best_bid_price]

    # ---------------- DEPTH VIEW ----------------

    def print_top_levels(self, levels=5):
        print("\n--- ORDER BOOK ---")

        print("ASKS:")
        for price in sorted(self.asks.keys())[:levels]:
            total_qty = sum(o.qty for o in self.asks[price])
            print(f"{price} -> {total_qty}")

        print("BIDS:")
        for price in sorted(self.bids.keys(), reverse=True)[:levels]:
            total_qty = sum(o.qty for o in self.bids[price])
            print(f"{price} -> {total_qty}")


Example Test Case 1 — Limit orders only

In [30]:
ob = OrderBook()
ob.add_limit_order("buy", 100, 10)
ob.add_limit_order("sell", 101, 15)
ob.print_top_levels()


--- ORDER BOOK ---
ASKS:
101 -> 15
BIDS:
100 -> 10


Example Test Case 2 — Limit order matching

In [31]:
ob = OrderBook()
ob.add_limit_order("sell", 101, 10)
ob.add_limit_order("buy", 101, 15)
ob.print_top_levels()
print("Trades:", ob.trades)



--- ORDER BOOK ---
ASKS:
BIDS:
101 -> 5
Trades: [('BUY', 101, 10)]


Example Test Case 3 — Market orders

In [32]:
ob = OrderBook()
ob.add_limit_order("sell", 101, 10)
ob.add_limit_order("sell", 102, 10)
ob.add_market_order("buy", 15)
ob.print_top_levels()
print("Trades:", ob.trades)



--- ORDER BOOK ---
ASKS:
102 -> 5
BIDS:
Trades: [('BUY', 101, 10), ('BUY', 102, 5)]
